<a href="https://colab.research.google.com/github/cric96/DL-exercise/blob/main/test_with_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Networks applied in Aggregate Computing
In this notebook, I tried to apply Neural Network and Recurrent Neural Networks (RNN) in the context of Aggregate Computing (AC).

## Model

Usually, RNN are trained in indipendent sequences. In case of Aggregate Computing, the temporal sequence are correlated with each other following a neighbourhood policy. 

The key idea here is:
- express the system as a graph. Offline we can imagine to access to the entire system
- in each time step, we aggregate data from the neighbours and then we use RNN to compute the right output

# Imports

In [1]:
import tensorflow as tf
from datetime import datetime
from numpy.random import seed

# Simple Graph

In this model, I model graph as:
$G = (N, E)$

Where, $N$ contains a feature vector and an output vector.

$E$ is expressed as adjacency matrix, so: $E_{i,j} = 1$ iff $i$ is neighbour of $j$.

Input is a matrix contains the concatenation of feature vector and output vector, something like: $ I(n) = <f(n), o(n)>$.
Where $<a, b, c>$ means the column-wise fector concatenation. 
For instance, giving a vector $a = [1, 2]$ and $b = [2, 3]$, $<a,b> = [1, 2, 2, 3]$.

The neighbour aggregation data can be computed leveraging matrix multiplication --- so increasing traning performance.
$$ reduction(E * F) $$
Where F contains all node features.
$reduction$ is a multiset function (e.g. summation, mean,...).

Pay attention, this model can be easly used in a decentralized situation.
Indeed, the data aggregation will be done without global adjency matrix but only retrieving neighbour data. The dense layer already works locally.

In [2]:
n = 50000
feature = tf.constant([
                     [1], 
                     [0],
                     [0],
                     [0],
                    ], dtype=tf.float32)
output = tf.constant([
                     [0], 
                     [n],
                     [n],
                     [n],
                    ], dtype=tf.float32)

input = tf.concat([feature, output], axis = 1)
input = input[:,tf.newaxis, :]

neigh = tf.constant([
                     [n, 1, n, n], 
                     [1, n, 1, n],
                     [n, 1, n, 1],
                     [n, n, 1, n],
                    ], dtype=tf.float32)

ground = tf.constant([
                     [1, 0], 
                     [0, 1],
                     [0, 2],
                     [0, 3],
                    ], dtype=tf.float32)

# Forward Pass

Taking the output of previous step, adjecency matrix, and feature vector, this function compute the value of the next evaluation.
So
1. compute neighbourhood feature via aggregation
2. concat neighbourhood feature with local feature and previous output
3. perform a forward pass of a neural network.


In [1]:
def forward(result, neigh, feature, model, log_enable=False):
  if(log_enable):
    print("Input = ", result)
  reshape = tf.reshape(result[:,:, 1], feature.shape[0])
  neigh_evaluation = tf.reduce_min(tf.multiply(neigh, reshape), 1) ## pass reduction strategy
  input_network = tf.concat([result[:,:, 0], neigh_evaluation[:, tf.newaxis]], 1)
  if(log_enable):
    print("Neighbour Aggregation = ", input_network[:, tf.newaxis])
  result = model(input_network[:, tf.newaxis])
  result = tf.reshape(result, feature.shape[:2]) ## adapt the shape in order to work both in linear model and in the recurrent model.
  result = tf.concat([feature, result], axis = 1)
  if(log_enable):
    print("Output = ", result)
  return tf.expand_dims(result, [1])

# Model creation
Create a sequential model given layers and the input shape

In [4]:
def create_model(input_shape, layers):
  input_layer = tf.keras.layers.InputLayer(input_shape=input.shape[1:], batch_size=input_shape[0])
  layers.insert(0, input_layer)
  return tf.keras.Sequential(layers)

## Recurrent model
Simple network creation with multiple recurrent layer 


In [5]:
def instantiate_layers():
  return [
    tf.keras.layers.GRU(units = 4, activation='relu', return_sequences=True, stateful=True),
    tf.keras.layers.GRU(units = 1, activation='relu', return_sequences=False, stateful=True, bias_initializer='ones'),
  ]

## Linear Model
Simple network with multiple linear layer

In [11]:
def instantiate_linear_layers():
  return [
    tf.keras.layers.Dense(units = 8, activation='relu'),
    tf.keras.layers.Dense(units = 4, activation='relu'),
    tf.keras.layers.Dense(units = 1, activation='relu', bias_initializer='ones')
  ]

## Model instatiation


In [12]:
linear = "linear"
recurrent = "recurrent"
same = "same"

def instatiate(input, mode = "same"):
  if mode == linear:
    return create_model(input.shape, instantiate_linear_layers())
  elif mode == recurrent:
    return create_model(input.shape, instantiate_layers())

def copy_model(model, mode = "same", input = ""):
  if mode == linear:
    return model
  elif mode == recurrent:
    change_model = instatiate(input, recurrent)
    change_model.set_weights(model.get_weights())
    return change_model
mode = linear
model = instatiate(input, linear)

# Train function

In [16]:
## TODO pass input and network
def train(model, forward_fn, data, iteration, stabilise_in, stabilisation_check, loss, optimizer, each=100):
  input, neigh, feature, ground = data
  for j in range(iteration):
    with tf.GradientTape() as tape:
      result = input
      to_backprop = 0
      for i in range(stabilise_in):
        result = forward_fn(result, neigh, feature, model)
      for i in range(stabilisation_check):
        result = forward_fn(result, neigh, feature, model)
        to_backprop += 1 / stabilisation_check * loss(ground, result[:, 0, :])
      model.reset_states()
      gradient = tape.gradient(to_backprop, model.weights)
      optimizer.apply_gradients(zip(gradient, model.weights))
    if(j % each == 0):
      print("Epoch ", j ,"Loss = ", tf.reduce_sum(to_backprop))  


## Train loop
Here I want only to find a function that overfits, so it solve this specific problem.


In [ ]:
#seed(42)
#tf.random.set_seed(42)
model = instatiate(input, mode) ## comment to avoid the recomputation of weights

iteration = 3000
stabilise_in = 4
stabilisation_check = 10

loss = tf.losses.mse
optimizer = tf.optimizers.Adam()
data = (input, neigh, feature, ground)
train(model, forward, data, iteration, stabilise_in, stabilisation_check, loss, optimizer, 50)

result = input
for i in range(10):
  result = forward(result, neigh, feature, model)
print(result)
model.reset_states()

tf.Tensor([    0. 50000. 50000. 50000.], shape=(4,), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor([1.3198751 1.        1.        1.       ], shape=(4,), dtype=float32)
tf.Tensor(1.3198751, shape=(), dtype=float32)
tf.Tensor([1.5296187 0.8180584 0.8180584 0.8180584], shape=(4,), dtype=float32)
tf.Tensor(1.5296187, shape=(), dtype=float32)
tf.Tensor([1.4922361  0.85116106 0.85116106 0.85116106], shape=(4,), dtype=float32)
tf.Tensor(1.4922361, shape=(), dtype=float32)
tf.Tensor([1.4992619 0.8451383 0.8451383 0.8451383], shape=(4,), dtype=float32)
tf.Tensor(1.4992619, shape=(), dtype=float32)
tf.Tensor([1.4979837 0.8462341 0.8462341 0.8462341], shape=(4,), dtype=float32)
tf.Tensor(1.4979837, shape=(), dtype=float32)
tf.Tensor([1.4982162 0.8460347 0.8460347 0.8460347], shape=(4,), dtype=float32)
tf.Tensor(1.4982162, shape=(), dtype=float32)
tf.Tensor([1.498174   0.84607106 0.84607106 0.84607106], shape=(4,), dtype=float32)
tf.Tensor(1.498174, shape=(), dtype=float32)
tf.

## Save model

In [15]:
model.save('model_dense_1')

INFO:tensorflow:Assets written to: model_dense_1/assets


# Check result, generalisation
In this part, I try to use the same network in another graph, to see if it can be used in different graphs.

## Linear network

In [20]:
n = 50000
feature_validation = tf.constant([
                     [0], 
                     [0],
                     [0],
                     [0],
                     [0]
                    ], dtype=tf.float32)
output_validation = tf.constant([
                     [0], 
                     [1],
                     [2],
                     [3],
                     [4]
                    ], dtype=tf.float32)
input_validation = tf.concat([feature_validation, output_validation], axis = 1)
input_validation = input_validation[:,tf.newaxis, :]
neigh_validation = tf.constant([
                     [n, 1, n, n, n], 
                     [1, n, 1, n, n],
                     [n, 1, n, 1, n],
                     [n, n, 1, n, 1],
                     [n, n, n, 1, n],
                    ], dtype=tf.float32)

change_model = copy_model(model, mode, input_validation)

result_validation = input_validation
model.reset_states()
for i in range(1000):
  result_validation = forward(result_validation, neigh_validation, feature_validation, change_model, False)
print(result_validation)

tf.Tensor(
[[[ 0. nan]]

 [[ 0. nan]]

 [[ 0. nan]]

 [[ 0. nan]]

 [[ 0. nan]]], shape=(5, 1, 2), dtype=float32)


## Square like

In [12]:
n = 50000
feature_validation = tf.constant([
                     [1], 
                     [0],
                     [0],
                     [0],
                    ], dtype=tf.float32)
output_validation = tf.constant([
                     [0], 
                     [n],
                     [n],
                     [n],
                    ], dtype=tf.float32)
input_validation = tf.concat([feature_validation, output_validation], axis = 1)
input_validation = input_validation[:,tf.newaxis, :]
neigh_validation = tf.constant([
                     [n, 1, 1, n], 
                     [1, n, n, 1],
                     [1, n, n, 1],
                     [n, 1, 1, n],
                    ], dtype=tf.float32)

change_model = copy_model(model, mode, input_validation)

result_validation = input_validation
model.reset_states() ## if has recurrent layers
for i in range(10):
  result_validation = forward(result_validation, neigh_validation, feature_validation, change_model, False)
print(result_validation)

tf.Tensor(
[[[1.         0.12369008]]

 [[0.         0.9757423 ]]

 [[0.         0.9757423 ]]

 [[0.         1.9846121 ]]], shape=(4, 1, 2), dtype=float32)


## Local test

In [14]:
print(model(tf.constant([[0, 100]]))) ## should be ~ 101
print(model(tf.constant([[0, 10]]))) ## should be ~ 11

tf.Tensor([[102.46612]], shape=(1, 1), dtype=float32)
tf.Tensor([[11.141671]], shape=(1, 1), dtype=float32)


# Advanced graph data
In this case, we suppose to have a collective state that will evolve accordingly the node execution

In [13]:
n = 50000
feature = tf.constant([[1],[0],[0],[0]], dtype=tf.float32)
output = tf.constant([[0],[n],[n],[n]], dtype=tf.float32)
state = tf.constant([[1],[1],[1],[1]], dtype=tf.float32)

input = tf.concat([feature, output, state], axis = 1)
input = input[:,tf.newaxis, :]

neigh = tf.constant([
                     [n, 1, n, n], 
                     [1, n, 1, n],
                     [n, 1, n, 1],
                     [n, n, 1, n],
                    ], dtype=tf.float32)

ground = tf.constant([[1, 0],[0, 1],[0, 2],[0, 3]], dtype=tf.float32)

## Forward pass rivisited


In [ ]:
def forward_with_state(result, neigh, feature, model, log_enable=False):
  if(log_enable):
    print("Input = ", result)
  reshape = tf.reshape(result[:,:, 1], feature.shape[0])
  neigh_output_evaluation = tf.reduce_min(tf.multiply(neigh, reshape[0]), 1) ## pass reduction strategy
  neigh_state_evaluation = tf.reduce_sum(tf.multiply(neigh, reshape[1]), 1) ## state evolution strategy
  input_network = tf.concat([result[:,:, 0], neigh_output_evaluation[:, tf.newaxis], neigh_state_evaluation[:, tf.newaxis]], 1)
  if(log_enable):
    print("Neighbour Aggregation = ", input_network[:, tf.newaxis])
  result = model(input_network[:, tf.newaxis])
  result = tf.reshape(result, feature.shape[:2]) ## adapt the shape in order to work both in linear model and in the recurrent model.
  result = tf.concat([feature, result], axis = 1)
  if(log_enable):
    print("Output = ", result)
  return tf.expand_dims(result, [1])